# Setup

## Installations and imports

### Installations
As some libraries that are not in the default version in colab are used, it is necessary to install them

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-geometric -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html

!pip install torchmetrics

### Imports
In the next snippet of code there are all the imports necessaries for the project and the tensorboard is initialized.

In [2]:
import plotly.graph_objects as go

import torch
from torch import nn
from torch.nn import functional as F

from torchmetrics import Accuracy

from torch_geometric.nn import GCNConv, global_max_pool
from torch_geometric.data import DataLoader
from torch_geometric.datasets import ModelNet
from torch_geometric.transforms import SamplePoints, NormalizeScale, KNNGraph, Compose

## Hyperparameters

In [3]:
hparams = {
    'bs': 1,
    'device': torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    'drive_root': '/content/drive/MyDrive/Dataset/ModelNet',
    'normalize_scale': True, 
    'fixed_num_of_points': 1024,
    'model_log': '/content/drive/MyDrive/GCN/GCN3_DFSc.pt', 
    'k': 3,
    'num_classes': 10,
    'level': 3,
    'dropout': 0.3
}

#Model

In [4]:
class GCN(nn.Module):
  def __init__(self, k=3, num_classes=10, level=3, dropout=0.3):
      super().__init__()
      self.k = k
      self.level = level
      self.classes = num_classes
      self.last_hidden_layer = 8 * 2**self.level

      self.conv1 = GCNConv(self.k, 16)
      self.conv2 = GCNConv(16, 32)
      self.conv3 = GCNConv(32, 64)
      self.fc = nn.Linear(self.last_hidden_layer, self.classes)

      self.bn1 = nn.BatchNorm1d(16)
      self.bn2 = nn.BatchNorm1d(32)
      self.bn3 = nn.BatchNorm1d(64)

      self.dropout = nn.Dropout(p=dropout) if dropout is not None else None

  def forward(self, x, edge_index, batch):
      # 1. Obtain node embeddings
      x = self.conv1(x, edge_index)
      x = self.dropout(x) if self.level == 1 and self.dropout is not None else x
      x = F.relu(self.bn1(x))

      if self.level >= 2:
          x = self.conv2(x, edge_index)
          x = self.dropout(x) if self.level == 2 and self.dropout is not None else x
          x = F.relu(self.bn2(x))

      if self.level >= 3:
          x = self.conv3(x, edge_index)
          x = self.dropout(x) if self.level == 3 and self.dropout is not None else x
          x = F.relu(self.bn3(x))

      # 2. Readout layer: Aggregate node embeddings into a unified graph embedding
      x = global_max_pool(x, batch)  # [batch_size=32, hidden_channels=64]

      # 3. Apply a final classifier
      x = self.fc(x)

      return x, F.softmax(x, dim=1)

# Dataset
First of all, it is necessary to make the drive folder with the dataset available to this collab in order not to download it every time. 

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Transformations
In this project is necessary to use some transformations to either normalize the data or to perform data augmentation.

In [6]:
def get_pre_transformation(number_points=1024):
    return SamplePoints(num=number_points)

def get_transformation(normalize_scale):
    if normalize_scale:  
      return Compose([NormalizeScale(), KNNGraph(k=9, loop=True, force_undirected=True)])
    else:
        return KNNGraph(k=9, loop=True, force_undirected=True)

## Test data

In [7]:
def get_dataset(root, number_points=1024, normalize_scale=True):
    
    test_dataset = ModelNet(root=root, name="10", train=False, pre_transform=get_pre_transformation(number_points), transform=get_transformation(normalize_scale))
    return test_dataset

## Helper functions
As there are some functionalities that are used by different functions or can be used in the future, a list of helpers fucntions has been created

In [8]:
# Method to visualize a cloud point
def visualize_point_cloud(point_cloud):
    edge_index, points, y = point_cloud
    edge_x = [], edge_y = [], edge_z = []
    edges_index = edge_index[1].numpy().T
    real_points = points[1].numpy()

    # Get coordinates from adjacency matrix
    for i, edge in enumerate(edges_index):
        x0, y0, z0 = real_points[edge[0]]
        x1, y1, z1 = real_points[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = [], node_y = [], node_z = []
    # Get node coordinates
    for node in real_points:
        x, y, z = node
        node_x.append(x)
        node_y.append(y)
        node_z.append(z)

    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # color scale options
            # 'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            # 'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            # 'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='<br>Network graph made with Python',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        annotations=[dict(
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002)],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()

# Testing inference

In [9]:
def test(test_data, model_state_dict_root):
    test_loader = DataLoader(test_data, batch_size=hparams['bs'], shuffle=False)
    model = GCN(hparams['k'], hparams['num_classes'], hparams['level'], hparams['dropout']).to(hparams['device'])

    accuracy = Accuracy(average='micro', compute_on_step=False).to(hparams['device'])
    model.load_state_dict(torch.load(model_state_dict_root))
    model.eval()
    # Metric stored information reset:
    accuracy.reset()
    # Batch loop for validation:
    with torch.no_grad():
        for i, data in enumerate(test_loader, 1):
            # Data retrieval from each bath:
            points = data.pos.to(hparams['device'])
            targets = data.y.to(hparams['device'])

            # Forward pass:
            preds, probs = model(points, data.edge_index.to(hparams['device']), data.batch.to(hparams['device']))  

            # Batch metrics calculation:
            accuracy.update(probs, targets)

    mean_accu = accuracy.compute().item()
    # Print of all metrics:
    print("Test Acc.: ", mean_accu)
    return mean_accu

In [10]:
test_dataset = get_dataset(hparams['drive_root'], hparams['fixed_num_of_points'], hparams['normalize_scale'])
test(test_dataset, hparams['model_log']) 

Test Acc.:  0.8777533173561096


0.8777533173561096